In [2]:
---

title: Lab 4 - Coffee Lovers Unite
author: Ben Laufer
format:
    html:
        code-fold: true
embed-resources: true

---

SyntaxError: invalid syntax (2775680049.py, line 1)

In [1]:
#libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

**Coffee Lovers Unite!**
If caffeine is one of the most popular drugs, then coffee is likely one of the most popular delivery systems for it. Aside from caffeine, people enjoy the wonderful variety of coffee-related drinks. Let’s do a rough investigation of the “market share” by two of the top coffee chains in the United States!

World Population Review provides some great data on store locations and chain prevalence. Check out this page for the Starbucks Coffee locations in the United States. Notice that this page only really gives the name of the state and the number of locations in that state.

**Scrape the Location Counts**

1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
- Starbucks
- Dunkin’ Donuts

2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [3]:
#1 (STARBUCKS)
#requests
sb_response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

#soup
sb_soup = BeautifulSoup(sb_response.content, "html.parser")


In [4]:
#table (dont want table to be a list)
sb_table = sb_soup.find("table",
                    attrs={
                      "class": "wpr-table"}
                  )

In [5]:
#turn html into actual dataframe

# initialize an empty list
rows = []

# iterate over all rows in the city table
#we do 1: b/c there is only 1 header row in this data
for state in sb_table.find_all("tr")[1:]:

    # Get all the cells (<th>) and (<td>) in the row.
    cells_state = state.find_all("th")
    cells_obs = state.find_all("td")
    
    # Find the the name of the state in cell[0]
    # which for most state is contained in the <a> tag
    state_tag = cells_state[0].find("a") or cells_state[0]
    state = state_tag.text.strip()

    #find # stores in 2023
    stores_2023_tag = cells_obs[0].find("td") or cells_obs[0]
    stores_2023 = stores_2023_tag.text.strip()

    #find # stores in 2023
    stores_2021_tag = cells_obs[1].find("td") or cells_obs[1]
    stores_2021 = stores_2021_tag.text.strip()

     #find # stores in 2023
    stores_2024_tag = cells_obs[2].find("td") or cells_obs[2]
    stores_2024 = stores_2024_tag.text.strip()

    # Append this data.
    rows.append({
            "state": state,
            "2023": stores_2023,
            "2021": stores_2021,
            "2024": stores_2024
     })


sb_df = pd.DataFrame(rows)
sb_df.head()


,state,2023,2021,2024
0,California,"3,080","2,959","3,117"
1,Texas,"1,346","1,215","1,409"
2,Florida,844,786,892
3,Washington,741,739,736
4,New York,692,643,715


In [6]:
#1 (DUNKIN DONUTS)
#requests
dd_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")

#soup
dd_soup = BeautifulSoup(dd_response.content, "html.parser")


In [7]:
#table (dont want table to be a list)
dd_table = dd_soup.find("table",
                    attrs={
                      "class": "wpr-table"}
                  )

In [8]:
#turn html into actual dataframe

# initialize an empty list
rows = []

# iterate over all rows in the city table
#we do 1: b/c there is only 1 header row in this data
for state in dd_table.find_all("tr")[1:]:

    # Get all the cells (<th>) and (<td>) in the row.
    cells_state = state.find_all("th")
    cells_obs = state.find_all("td")
    
    # Find the the name of the state in cell[0]
    # which for most state is contained in the <a> tag
    state_tag = cells_state[0].find("a") or cells_state[0]
    state = state_tag.text.strip()

    #find # stores in 2024
    stores_2024_tag = cells_obs[0].find("td") or cells_obs[0]
    stores_2024 = stores_2024_tag.text.strip()

    #find # stores in 2023
    stores_2023_tag = cells_obs[1].find("td") or cells_obs[1]
    stores_2023 = stores_2023_tag.text.strip()


    # Append this data.
    rows.append({
            "state": state,
            "2024": stores_2024,
            "2023": stores_2023
     })


dd_df = pd.DataFrame(rows)
dd_df.head()


,state,2024,2023
0,New York,"1,431","1,414"
1,Massachusetts,"1,042","1,068"
2,Florida,909,883
3,New Jersey,872,866
4,Illinois,711,692


In [9]:
#2. setup data
#will want to pivot for year and val because they are not fully overlap

#Starbucks
#pivot long, vars will be year to val
sb_long = pd.melt(sb_df, id_vars='state', value_vars=['2021', '2023', '2024'], var_name = 'year', value_name = 'num_stores')

#create brand var so that we know which store the obs is for
sb_long['Brand'] = 'Starbucks'


#Dunkin Donut
#pivot long
#pivot long, vars will be year to val
dd_long = pd.melt(dd_df, id_vars='state', value_vars=['2023', '2024'], var_name = 'year', value_name = 'num_stores')

#create brand var so that we know which store the obs is for
dd_long['Brand'] = 'Dunkin Donut'

#stack data using concat
df_stacked = pd.concat([sb_long, dd_long], axis=0, ignore_index=True)

**Supplemental Data**

4. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

5. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

6. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [10]:
#3 (wikipedia)
#requests
wiki_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")

#soup
wiki_soup = BeautifulSoup(wiki_response.content, "html.parser")


In [11]:
tables = wiki_soup.find_all("table")
len(tables)
#turn into just non-list
wiki_table = tables[0]

In [12]:
#DOESNT WORK, LINE ABOVE DOES
#table (dont want table to be a list)
#wiki_table = wiki_soup.find("table",
                    #attrs={
                      #"class": "wikitable sortable jquery-tablesorter"}
                  #)

In [13]:
#turn html into actual dataframe

# initialize an empty list
rows = []

# iterate over all rows in the city table
#we do 1: b/c there is only 1 header row in this data
for state in wiki_table.find_all("tr")[1:]:

    # Get all the cells (<td>) in the row.
    cells = state.find_all("td")
    
    # can just look at table and see what obs we want along and pick that indexes b/c its in a weirdish order
    state_tag = cells[2]
    state = state_tag.text.strip()

    #find # stores in 2024
    census_2020_tag = cells[3]
    census_2020 = census_2020_tag.text.strip()

    #find # stores in 2023
    census_2010_tag = cells[4]
    census_2010 = census_2010_tag.text.strip()


    # Append this data.
    rows.append({
            "state": state,
            "census_2020": census_2020,
            "census_2010": census_2010
     })


wiki_df = pd.DataFrame(rows)
wiki_df.head()


,state,census_2020,census_2010
0,California,"39,538,223","37,253,956"
1,Texas,"30,145,505","25,145,561"
2,Florida,"21,538,187","18,801,310"
3,New York,"20,201,249","19,378,102"
4,Pennsylvania,"13,002,700","12,702,379"


In [14]:
#Merge this wiki_df to our df_stacked with id as state
#use outer to ensure we don't lose any data (can change that later if we need to)
merged_df = pd.merge(wiki_df, df_stacked, how='outer', on='state')

In [15]:
#IS IT OKAY IF REVENUE ISN'T BY YEAR?
#5.
#I had chat GPT make an example dataframe of total revenue (where it is adjusted by population)
# Here is the code directly copied that it gave me:

#will be the same regardless of year
#so this dataframe displays the brand's overall revenue per state

# List of all states
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
          'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
          'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 
          'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
          'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
          'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
          'Wisconsin', 'Wyoming']

# Adjusted revenue data for Starbucks (based on larger population and presence)
starbucks_revenue = [600, 50, 550, 180, 2500, 800, 400, 150, 1600, 1300, 100, 120, 1100, 700, 350,
                     300, 500, 420, 80, 750, 1400, 900, 600, 200, 550, 90, 150, 300, 120, 850,
                     180, 1800, 1300, 80, 1100, 250, 550, 1100, 90, 480, 100, 950, 2100, 250, 
                     90, 900, 1600, 150, 800, 40]

sb_revenue = pd.DataFrame({
    'state': states,
    'revenue': starbucks_revenue})

sb_revenue['Brand'] = 'Starbucks'

# Adjusted revenue data for Dunkin' Donuts (stronger presence in the Northeast)
dunkin_donuts_revenue = [150, 10, 100, 80, 300, 150, 450, 80, 500, 300, 20, 60, 450, 250, 80,
                         100, 200, 180, 50, 400, 1200, 500, 250, 80, 220, 40, 80, 150, 90, 700,
                         120, 1000, 500, 30, 600, 120, 150, 700, 80, 220, 60, 400, 800, 120, 
                         30, 500, 700, 90, 400, 20]

dd_revenue = pd.DataFrame({
    'state': states,
    'revenue': dunkin_donuts_revenue})

dd_revenue['Brand'] = 'Dunkin Donut'

#stack data using concat
revenue_stacked = pd.concat([sb_revenue, dd_revenue], axis=0, ignore_index=True)



In [16]:
#merge with our merged_df
#use outer to ensure we don't lose any data (can change that later if we need to)
merged_df = pd.merge(merged_df, revenue_stacked, how='outer', on=['state', 'Brand'])

In [17]:
#6.
#used chat GPT to automate this part of the task
#lists all the states with their corresponding region
regions = {
    "Northeast": ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont", "New Jersey", "New York", "Pennsylvania"],
    "Midwest": ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin", "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"],
    "South": ["Delaware", "Florida", "Georgia", "Maryland", "North Carolina", "South Carolina", "Virginia", "Washington, D.C.", "West Virginia", "Alabama", "Kentucky", "Mississippi", "Tennessee", "Arkansas", "Louisiana", "Oklahoma", "Texas"],
    "West": ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah", "Wyoming", "Alaska", "California", "Hawaii", "Oregon", "Washington"]
}

# creates df so that it will be easy to merge the data
#it creates the dataframe with state, and region. Loops for each region and then gets all the states inside it.
# For each state its creating a "tuple" where the first element is the state and the second is the region
region_df = pd.DataFrame([(state, region) for region, states in regions.items() for state in states], columns=['state', 'region'])


#Merge with merged_df on state
final_df = pd.merge(merged_df, region_df, how='outer', on=['state'])